In [1]:
import numpy as np
from dog_breed_detector_model import DogBreedDetectorModel
from data_extractor import DataExtractor
from tensorflow.python.keras.applications.densenet import DenseNet201
from tensorflow.python.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.python.keras.models import Sequential, Model

/home/vetal/anaconda3/envs/aind2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_extractor = DataExtractor('dog_images/train', 'dog_images/valid', 'dog_images_test')

train_data = data_extractor.load_train_data()
train_labels, train_num_dog_breeds = data_extractor.load_train_labels()

valid_data = data_extractor.load_valid_data()
valid_labels, valid_num_dog_breeds = data_extractor.load_valid_labels()

assert train_num_dog_breeds == valid_num_dog_breeds

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.


In [ ]:
cnn_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=cnn_model.output_shape[1:]))
top_model.add(Dense(133, activation='softmax'))
top_model.load_weights('saved_model/best_model.h5')

In [ ]:
print(top_model.summary())

In [ ]:
final_model = Model(input=cnn_model.input, output=top_model(cnn_model.output))
print(final_model.summary())

In [3]:
from dog_breed_detector_model import DogBreedDetectorModel

model = DogBreedDetectorModel(num_dog_breeds=train_num_dog_breeds)
model.init_model(train_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               255493    
Total params: 255,493
Trainable params: 255,493
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.train(train_data, train_labels, valid_data, valid_labels)

In [4]:
from glob import glob
dog_breed_names = [item[21:-1] for item in sorted(glob('{}/*/'.format('dog_images/train')))]
print(dog_breed_names)

['Affenpinscher', 'Afghan_hound', 'Airedale_terrier', 'Akita', 'Alaskan_malamute', 'American_eskimo_dog', 'American_foxhound', 'American_staffordshire_terrier', 'American_water_spaniel', 'Anatolian_shepherd_dog', 'Australian_cattle_dog', 'Australian_shepherd', 'Australian_terrier', 'Basenji', 'Basset_hound', 'Beagle', 'Bearded_collie', 'Beauceron', 'Bedlington_terrier', 'Belgian_malinois', 'Belgian_sheepdog', 'Belgian_tervuren', 'Bernese_mountain_dog', 'Bichon_frise', 'Black_and_tan_coonhound', 'Black_russian_terrier', 'Bloodhound', 'Bluetick_coonhound', 'Border_collie', 'Border_terrier', 'Borzoi', 'Boston_terrier', 'Bouvier_des_flandres', 'Boxer', 'Boykin_spaniel', 'Briard', 'Brittany', 'Brussels_griffon', 'Bull_terrier', 'Bulldog', 'Bullmastiff', 'Cairn_terrier', 'Canaan_dog', 'Cane_corso', 'Cardigan_welsh_corgi', 'Cavalier_king_charles_spaniel', 'Chesapeake_bay_retriever', 'Chihuahua', 'Chinese_crested', 'Chinese_shar-pei', 'Chow_chow', 'Clumber_spaniel', 'Cocker_spaniel', 'Collie',

In [5]:
from final_model import FinalModel

model.load_model()
fin_model = FinalModel(model.model, dog_breed_names)

Model weights loaded from saved_model/best_model.h5


In [6]:
import tensorflow as tf

tf_estimator = tf.keras.estimator.model_to_estimator(keras_model=fin_model.final_model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_summary_steps': 100, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_session_config': None, '_model_dir': '/tmp/tmpxq5jxxca', '_save_checkpoints_steps': None, '_global_id_in_cluster': 0, '_task_id': 0, '_is_chief': True, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa91010ef60>, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_evaluation_master': ''}


In [7]:
from tensorflow.python.keras.applications.densenet import preprocess_input

def preprocess_image(image_buffer, image_shape=(224, 224, 3)):
    '''
    Preprocess JPEG encoded bytes to 3D float Tensor and rescales
    it so that pixels are in a range of [-1, 1]

    :param image_buffer: Buffer that contains JPEG image
    :return: 4D image tensor (1, width, height,channels) with pixels scaled
             to [-1, 1]. First dimension is a batch size (1 is our case)
    '''

    # Decode the string as an RGB JPEG.
    # Note that the resulting image contains an unknown height and width
    # that is set dynamically by decode_jpeg. In other words, the height
    # and width of image is unknown at compile-time.
    image_buffer = tf.reshape(image_buffer, [])
    img = tf.image.decode_jpeg(image_buffer, channels=3)
    
    # After this point, all image pixels reside in [0,1)
    # until the very end, when they're rescaled to (-1, 1).  The various
    # adjust_* ops all require this range for dtype float.
    # img = tf.image.convert_image_dtype(img, dtype=tf.float32)

    # Networks accept images in batches.
    # The first dimension usually represents the batch size.
    # In our case the batch size is one.
    img = tf.expand_dims(img, 0)
    img = tf.image.resize_images(img, size=(image_shape[1], image_shape[0]))
    
    # Finally, rescale to [-1,1] instead of [0, 1)
    # img = tf.subtract(img, 0.5)
    # img = tf.multiply(img, 2.0)
    img = preprocess_input(img)
    
    return img

def serving_input_receiver_fn():
    '''
    An input receiver that expects a serialized tf.Example

    :return: a return type for a serving_input_receiver_fn
    '''
    serialized_tf_example = tf.placeholder(tf.string, shape=[1], name='input_image')
    feature_configs = {
        'image/encoded': tf.FixedLenFeature(
            shape=[], dtype=tf.string),
    }
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    jpegs = tf_example['image/encoded']
    
    receiver_tensors = {'examples': jpegs}
    images = tf.map_fn(preprocess_image, jpegs, dtype=tf.float32)
    features = {'input_2': tf.squeeze(images, [0])}
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [8]:
print(fin_model.final_model.input_names)
print(fin_model.final_model.output_names)

['input_2']
['sequential_1']


In [8]:
serving_input_receiver = serving_input_receiver_fn()

In [9]:
print(serving_input_receiver.receiver_tensors)
print(serving_input_receiver.receiver_tensors_alternatives)
print(serving_input_receiver.features)

{'examples': <tf.Tensor 'ParseExample/ParseExample:0' shape=(1,) dtype=string>}
None
{'input_2': <tf.Tensor 'Squeeze:0' shape=(1, 224, 224, 3) dtype=float32>}


In [10]:
estimator_spec = tf_estimator.model_fn(
    features=serving_input_receiver.features,
    labels=None,
    mode='infer',
    config=tf_estimator.config)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


In [15]:
print(estimator_spec.export_outputs)

{'serving_default': <tensorflow.python.estimator.export.export_output.PredictOutput object at 0x7f966dd14630>}


In [11]:
signature = estimator_spec.export_outputs['serving_default'].as_signature_def(serving_input_receiver.receiver_tensors)

In [12]:
print(signature)

inputs {
  key: "examples"
  value {
    name: "ParseExample/ParseExample:0"
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "sequential_2"
  value {
    name: "sequential_2/dense_1/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 133
      }
    }
  }
}
method_name: "tensorflow/serving/predict"



In [8]:
tf_estimator.export_savedmodel(
    'export_model/',
    serving_input_receiver_fn,
    strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Restoring parameters from /tmp/tmpxq5jxxca/keras_model.ckpt
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"export_model/temp-b'1527440644'/saved_model.pb"


b'export_model/1527440644'

In [6]:
print(fin_model.final_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [7]:
dog_breed = fin_model.predict('dog_images/test/004.Akita/Akita_00258.jpg')
print(dog_breed)

Predicting on dog_images/test/004.Akita/Akita_00258.jpg
Akita


In [ ]:
fin_model.export_for_serving()